# Google Colaboratory

In [ ]:
# if you use colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# If you use colab, this is colab_path, please change it to your own path to the code folder of this project
# WARNING: IF YOU ARE USING COLAB, THIS COULD ONLY BE RUN ONCE! OTHERWISE NOTEBOOK WILL CRASH
%cd /content/drive/MyDrive/chuangjl/code

/content/drive/MyDrive/chuangjl/code


# Configurations

In [ ]:
EMBEDDING_MODEL_NAME = "thenlper/gte-large" #"BAAI/bge-large-en-v1.5"
READER_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

data_base_path = "../database"

# Academics
handbooks_path = "Academics_LTI/Handbook"
program_path = "Academics_LTI/Programs_and_staff"
# Course
schedule_fall23_path = "Courses_CMU/Schedule/augmentFall23.txt"
schedule_spring24_path = "Courses_CMU/Schedule/augmentSpring24.txt"
schedule_summer24_path = "Courses_CMU/Schedule/augmentSummer24.txt"

calendar_2324_path = "Courses_CMU/Calendars/Calendar_scentences_2324.txt"
calendar_2425_path = "Courses_CMU/Calendars/Calendar_scentences_2425.txt"
# Events
commencement_path = "Events_CMU/Commencement/Commencement.txt"
spring_path = "Events_CMU/Spring/carnival_and_reunion.txt"
# Faculty
faculty_path = "Faculty_LTI/Faculty/LTI_faculty.txt"
researches_path = "Faculty_LTI/Research/"
# History
History_path = "History"

# Import Dependencies

In [ ]:
!pip install -q ragatouille

In [ ]:
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00


In [ ]:
!pip install -q datasets

In [ ]:
# Miscellany
from tqdm.notebook import tqdm
import pandas as pd
import torch
from typing import Optional, List, Tuple
import os
import matplotlib.pyplot as plt

import locale
from ragatouille import RAGPretrainedModel

# langchain
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.text_splitter import RecursiveCharacterTextSplitter

# datasets
import datasets
from datasets import concatenate_datasets
from datasets import Dataset

# transformers
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Pipeline
from transformers import pipeline

# Initialize Embedding and Reader Model

In [ ]:
from huggingface_hub import login
login()

In [ ]:
embedding_tokenizer =  AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
embedding_model = HuggingFaceEmbeddings(
    model_name = EMBEDDING_MODEL_NAME,
    multi_process = True,
    model_kwargs = {"device": "cuda"},
    encode_kwargs = {"normalize_embeddings": True},  # set True for cosine similarity
)

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# quantization parameters
# we will use 4 bit quantization to reduce model size
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization by replacing the Linear layers with FP4/NF4 layers from bitsandbytes.
    bnb_4bit_quant_type="nf4", # This sets the quantization data type in the bnb.nn.Linear4Bit layers. Options are FP4 and NF4 data types which are specified by fp4 or nf4.
    bnb_4bit_use_double_quant=True, # False # nested quantization where the quantization constants from the first quantization are quantized again.
    bnb_4bit_compute_dtype=torch.bfloat16, # This sets the computational type which might be different than the input time. For example, inputs might be fp32, but computation can be set to bf16 for speedups.
)

reader_tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
reader_model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Documents & Query Embedder

In [ ]:
def get_line_to_dataset(file_path, num_line_combine = 10) -> dict:
    filename = os.path.basename(file_path).split('/')[-1]
    text = []
    source = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            text.append(line)
            source.append(f"{filename}")

    result = {"text": text, "source": source}
    return result

In [ ]:
def read_as_list(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    return lines

def concatenate_strings(string_list):
    # Concatenate all strings in the list
    result = ' '.join(string_list)
    return result

def list_to_dict(string_list, path):
    result = {"text": [string_list], "source": [os.path.basename(path).split('/')[-1]]}
    return result

In [ ]:
def expand_ds(line_by_line_file_paths: List, whole_file_paths: List):
  ds = Dataset.from_dict({"text": [], "source": []})
  # if each line of the file is considered as a document
  for path in line_by_line_file_paths:
      file_path = os.path.join(data_base_path, path)

      # if the path is a directory
      if os.path.isdir(file_path):
          for file in os.listdir(file_path):
              folder_file_path = os.path.join(file_path, file)
              temp_dict = get_line_to_dataset(folder_file_path)
              temp_ds = Dataset.from_dict(temp_dict)
              ds = concatenate_datasets([ds, temp_ds], axis = 0)

      # if the path is a file
      else:
          temp_dict = get_line_to_dataset(file_path)
          temp_ds = Dataset.from_dict(temp_dict)
          ds = concatenate_datasets([ds, temp_ds], axis = 0)

  # if the entire file is considered as a document
  for path in whole_file_paths:
      file_path = os.path.join(data_base_path, path)

      # if the path is a directory
      if os.path.isdir(file_path):
          for file in os.listdir(file_path):
              folder_file_path = os.path.join(file_path, file)
              temp_list = read_as_list(folder_file_path)
              string_list = concatenate_strings(temp_list)
              temp_dict = list_to_dict(string_list, folder_file_path)
              temp_ds = Dataset.from_dict(temp_dict)
              ds = concatenate_datasets([ds, temp_ds], axis = 0)

      # if the path is a file
      else:
          temp_list = read_as_list(file_path)
          string_list = concatenate_strings(temp_list)
          temp_dict = list_to_dict(string_list, file_path)
          temp_ds = Dataset.from_dict(temp_dict)
          ds = concatenate_datasets([ds, temp_ds], axis = 0)
  return ds

In [ ]:
# line_by_line_file_paths = [schedule_fall23_path,
#                            schedule_spring24_path,
#                            schedule_summer24_path,
#                            calendar_2324_path,
#                            calendar_2425_path,
#                            researches_path]
# whole_file_paths = [commencement_path,
#                     spring_path,
#                     faculty_path,
#                     program_path,
#                     handbooks_path,
#                     History_path]

# ds = expand_ds(line_by_line_file_paths, whole_file_paths)

# if you want to save some time, or that the path does not work properly.
ds = datasets.load_dataset("ge0rgeli/CMU_LTI")
ds = ds['train']

Generating train split:   0%|          | 0/12937 [00:00<?, ? examples/s]

## Embedding and Preprocessing

In [ ]:
raw_documents = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]

  0%|          | 0/12937 [00:00<?, ?it/s]

In [ ]:
def recursive_split_documents_by_characters(
    chunk_size,
    chunk_overlap,
    raw_documents,
    tokenizer):
    # text splitter
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=chunk_size,
        chunk_overlap = chunk_overlap,
        add_start_index=True,
        strip_whitespace=True,
    )

    # split documents
    processed_documents = []
    for doc in raw_documents:
        processed_documents += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    unique_processed_documents = []
    for doc in processed_documents:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            unique_processed_documents.append(doc)

    return unique_processed_documents

In [ ]:
# ~30s
processed_documents = recursive_split_documents_by_characters(
    chunk_size = 512,  # We choose a chunk size adapted to our model
    chunk_overlap = 50,
    raw_documents = raw_documents,
    tokenizer = embedding_tokenizer,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors


# Retriever


Note: This will take about 1 minute if you are running with V100 + High-RAM

In [ ]:
# ~3 mins
database = FAISS.from_documents(processed_documents,
                                embedding_model,
                                distance_strategy=DistanceStrategy.COSINE)

# LLM as Retriever

*Note: This module is written during experiment, and was no longer in use since its sub-optimal performace.*

## Build independent datasets  

In [ ]:
# acadmics_ds = expand_ds([], [handbooks_path, program_path])
# courses_ds = expand_ds([schedule_fall23_path,
#                            schedule_spring24_path,
#                            schedule_summer24_path,
#                            calendar_2324_path,
#                            calendar_2425_path], [])
# events_ds = expand_ds([], [commencement_path, spring_path])
# faculty_ds = expand_ds([researches_path], [faculty_path])
# history_ds = expand_ds([], [History_path])

## Build independent knowledge base

In [ ]:
# def generate_knowledge_base(ds):
#   return [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]

In [ ]:
# acadmics_knowledge_base = generate_knowledge_base(acadmics_ds)
# course_knowledge_base = generate_knowledge_base(courses_ds)
# event_knowledge_base = generate_knowledge_base(events_ds)
# faculty_knowledge_base = generate_knowledge_base(faculty_ds)
# history_knowledge_base = generate_knowledge_base(history_ds)

## Process knowledge base

In [ ]:
# acadmics_knowledge = recursive_split_documents_by_characters(
#     512,  # We choose a chunk size adapted to our model
#     acadmics_knowledge_base,
#     tokenizer_name=EMBEDDING_MODEL_NAME,
# )
# course_knowledge = recursive_split_documents_by_characters(
#     512,  # We choose a chunk size adapted to our model
#     course_knowledge_base,
#     tokenizer_name=EMBEDDING_MODEL_NAME,
# )
# event_knowledge = recursive_split_documents_by_characters(
#     512,  # We choose a chunk size adapted to our model
#     event_knowledge_base,
#     tokenizer_name=EMBEDDING_MODEL_NAME,
# )
# faculty_knowledge = recursive_split_documents_by_characters(
#     512,  # We choose a chunk size adapted to our model
#     faculty_knowledge_base,
#     tokenizer_name=EMBEDDING_MODEL_NAME,
# )
# history_knowledge = recursive_split_documents_by_characters(
#     512,  # We choose a chunk size adapted to our model
#     history_knowledge_base,
#     tokenizer_name=EMBEDDING_MODEL_NAME,
# )

## Build independent vector database

In [ ]:
# acadmics_vd = FAISS.from_documents(
#     acadmics_knowledge, embedding_model, distance_strategy=DistanceStrategy.COSINE
# )
# course_vd = FAISS.from_documents(
#     course_knowledge, embedding_model, distance_strategy=DistanceStrategy.COSINE
# )
# event_vd = FAISS.from_documents(
#     event_knowledge, embedding_model, distance_strategy=DistanceStrategy.COSINE
# )
# faculty_vd = FAISS.from_documents(
#     faculty_knowledge, embedding_model, distance_strategy=DistanceStrategy.COSINE
# )
# history_vd = FAISS.from_documents(
#     history_knowledge, embedding_model, distance_strategy=DistanceStrategy.COSINE
# )

## QA Model as Retriever

In [ ]:
# # Use QA model
# llm_retriever = pipeline(model="deepset/roberta-base-squad2")

In [ ]:
# def get_document_num(response: str):
#   if "Class 1" in response or "class 1" in response: return 1
#   if "Class 2" in response or "class 2" in response: return 2
#   if "Class 3" in response or "class 3" in response: return 3
#   if "Class 4" in response or "class 4" in response: return 4
#   if "Class 5" in response or "class 5" in response: return 5
#   return 0

In [ ]:
# def get_knowledge_base(document_num):
#   if document_num == 1:
#     knowledge = acadmics_vd
#   if document_num == 2:
#     knowledge = course_vd
#   if document_num == 3:
#     knowledge = event_vd
#   if document_num == 4:
#     knowledge = faculty_vd
#   if document_num == 5:
#     knowledge = history_vd
#   if document_num == 0:
#     knowledge = None
#   return knowledge

# Reranking

In [ ]:
# prevent encoding error
locale.getpreferredencoding = lambda: "UTF-8"
rerank_model = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Reader Pipeline

In [ ]:
reader_pipeline = pipeline(
    model = reader_model,
    tokenizer = reader_tokenizer,
    task = "text-generation",
    do_sample = True,
    temperature = 0.1,
    repetition_penalty = 1.1,
    return_full_text = False,
    max_new_tokens = 700,
)

# Few-Shot Prompting

In [ ]:
few_shot_prompt = """Question: Is summer internship between the first and second year necessary for students in MSAII programs?
Answer: Yes
Question: In general, how many units of graduate-level courses must be passed for Ph.D. in Language and Information Technologies?
Answer: 96
Question: What is the date of 2025 Summer One & Mini-5 voucher deadline?
Answer: June 18, Wednesday in Summer 2025
Question: Who is the instructor of 11711 in Spring 2024
Answer:  Neubig
Question: When is Commencement Ceremony?
Answer: 10–11:30 a.m, Sunday, May 12
Question: What time of the day is Kiltie Band Concert?
Answer: 3:30 PM-4:30 PM ET
Question: What is Daphne Ippolito's Email?
Answer: daphnei@cmu.edu
Question:  Who is the author of the paper 'StyleRF: Zero-Shot 3D Style Transfer of Neural Radiance Fields'?
Answer: Eric P. Xing
Question: Who in 1986 wrote a white paper proposing the creation of "a School of Computer Science?
Answer: Nico Habermann and then-CMU provost Angel Jordan
Question: In what month do students and alumni reviewed potential mascot images in focus groups?
Answer: October"""

# Customized Prompt

In [ ]:
# sys_prompt = [
#     {
#         "role": "system",
#         "content": """Below are some examples of actual questions and answers pair.
# {few_shot_prompt}

# Please answer the question using only the information in the context. Above are some examples of actual questions and answers. Your answer should have the similar format as the example answers. Do not generate sentences, generate keywords only. Your answer should be straightforward and as short as possible. Your answer should not include the source of the information or context. If the answer cannot be deduced from the context, do not give answer, just say that you don't know.""",
#     },
#     {
#         "role": "user",
#         "content": """Context:
# {context}
# Here is the question you need to answer.
# Question: {question}""",
#     },
# ]

sys_prompt = [
    {"role": "user", "content": """Below are some examples of questions and answers pair. {few_shot_prompt}.
I want you to make a summary about the format of these answers""" },
    {"role": "assistant", "content": "These examples demonstrate the format of reference questions. The answers to these questions provide clear, concise responses that directly address the questions asked. The answer contains mostly keywords but not complete sentences. The answers does not include any explanation." },
    {"role": "user", "content": "Now, here are some context documents. Context: {context}Based on these context documents, could you answer the following question? Question: {question}. Your answer should have the same format as the example answers, as you just summarized. Do not give any other information other than the answer!"}
]

sys_prompt = [
    {"role": "user", "content": """Below are some documents as context {context}.""" },
    {"role": "assistant", "content": "Understood" },
    {"role": "user", "content": """Below are some example questions and their answers as reference {few_shot_prompt}.""" },
    {"role": "assistant", "content": "Understood" },
    {"role": "user", "content": "Based on these context documents, could you answer the following question? Question: {question}. Your answer should have the same format as the example answers. Your answer should not have any prefix or suffix. Your answer should not have any other information, including explanation, process, note, and so on. Do not give any other information other than the answer!"},
]

prompt_template = reader_tokenizer.apply_chat_template(
    sys_prompt, tokenize=False, add_generation_prompt=True
)
print(prompt_template)

<s>[INST] Below are some documents as context {context}. [/INST] Understood </s><s>[INST] Below are some example questions and their answers as reference {few_shot_prompt}. [/INST] Understood </s><s>[INST] Based on these context documents, could you answer the following question? Question: {question}. Your answer should have the same format as the example answers. Your answer should not have any prefix or suffix. Your answer should not have any other information, including explanation, process, note, and so on. Do not give any other information other than the answer! [/INST]


# RAG Pipeline

In [ ]:
def rag(
    question, llm, database,
    rerank_model = None,
    num_retrieved_docs = 40,
    num_return_docs = 5,
    few_shot_prompt = few_shot_prompt,
    print_output = True):
    # Gather documents with retriever
    if print_output: print("retrieving documents")
    rel_docs = database.similarity_search(query = question, k = num_retrieved_docs)
    rel_docs = [doc.page_content for doc in rel_docs]  # keep only the text

    # rerank results
    if rerank_model:
        if print_output: print("reranking")
        rel_docs = rerank_model.rerank(question, rel_docs, k = num_return_docs)
        rel_docs = [doc["content"] for doc in rel_docs]

    # select top k
    rel_docs = rel_docs[:num_return_docs]

    # build the final prompt
    context = "".join([f"Document {str(i)}: " + doc + "\n" for i, doc in enumerate(rel_docs)])
    final_prompt = prompt_template.format(few_shot_prompt = few_shot_prompt,
                                          context = context,
                                          question = question)

    # generating an answer
    if print_output: print("generating answer")
    answer = llm(final_prompt, pad_token_id=llm.tokenizer.eos_token_id)[0]["generated_text"]

    return final_prompt, answer

In [ ]:
# question_template = "If I give you the following question: '{question}' Which type of documents do you think would answer the question? Please answer with only one number of the class of document, without using complete sentences. If you are not sure what is the answer, answer 0."
# context="I have a bunch of documents about Carnegie Mellon University(CMU) and Language Technology Institute (LTI), which is a department of CMU. They are generally divided into 5 classes: Class 1 documents are about LTI, including an information about undergrad programs, master programs(MLT, MSAII, MCDS, MIIS). They also include information about dual Ph.D and Ph.D. Handbook about these programs are included. The documents also include information about LTI staff (name, position) Class 2 documents are about courses (title, number, instructor, classroom, etc) and calendars (beginning of semester) of CMU. Class 3 documents are about time and arrangement of events of CMU, including Spring Carnival, Reunion, as well as the commencement. Class 4 documents about LTI faculties and their researches, if you were asked about terms you are unfamiliar with, this is where you might find the answers. Class 5 documents are about history of the university. They include information about 25 great things about CMU, history of School of Computer Science (SCS), The Kiltie Band, Buggy Races, facts about CMU, and Tartan."

# def rag_with_llm(
#     question, llm, database,
#     rerank_model = None,
#     num_retrieved_docs = 40,
#     num_return_docs = 5,
#     few_shot_prompt = few_shot_prompt,
#     print_output = True):
#     # Gather documents with retriever
#     if print_output: print("retrieving documents")
#     final_question = question_template.format(question = question)
#     response = llm_retriever(question = final_question, context=context)
#     document_num = # get_document_num(response = response['answer'])
#     if print_output: print("retrieved document_num: ", document_num)
#     knowledge = get_knowledge_base(document_num)
#     relevant_docs = knowledge.similarity_search(query=question, k = num_retrieved_docs)
#     relevant_docs = [doc.page_content for doc in relevant_docs]
#     rel_docs = database.similarity_search(query = question, k = num_retrieved_docs)
#     rel_docs = [doc.page_content for doc in rel_docs]  # keep only the text

#     # rerank results
#     if rerank_model:
#         if print_output: print("reranking")
#         rel_docs = rerank_model.rerank(question, rel_docs, k = num_return_docs)
#         rel_docs = [doc["content"] for doc in rel_docs]

#     # select top k
#     rel_docs = rel_docs[:num_return_docs]

#     # build the final prompt
#     context = "".join([f"Document {str(i)}: " + doc + "\n" for i, doc in enumerate(rel_docs)])
#     final_prompt = prompt_template.format(few_shot_prompt = few_shot_prompt,
#                                           context = context,
#                                           question = question)

#     # generating an answer
#     if print_output: print("generating answer")
#     answer = llm(final_prompt, pad_token_id=llm.tokenizer.eos_token_id)[0]["generated_text"]

#     return final_prompt, answer

## Test

In [ ]:
# Usually takes about 30 - 40s, takes about 1 min if the document is large

question = "When is the class 11711 being held in spring 2024?"
final_prompt, answer = rag(question = question,
                                      llm = reader_pipeline,
                                      database = database,
                                      rerank_model = rerank_model,
                                      print_output = True)

retrieving documents
reranking


100%|██████████| 2/2 [00:00<00:00, 29.42it/s]


generating answer


In [ ]:
print("==================================Question================================")
print(question)
print("==================================Context=================================")
print(final_prompt)
print("==================================Answer==================================")
print(answer)

==================================Question================================
When is the class 11711 being held in spring 2024?
==================================Context=================================
<s>[INST] Below are some documents as context Document 0: Spring 2024, 11711: Below is information about CMU Spring 2024 course 11711: Advanced Natural Language Processing. The course number is 11711 or 11-711. The course Advanced Natural Language Processing(11711) in spring 2024 has 12.0 unit. The professor Neubig is teaching the course Advanced Natural Language Processing(11711) in spring 2024. The professor/instructor of Advanced Natural Language Processing(11711) in spring 2024 is Neubig. The department of Advanced Natural Language Processing(11711) is Language Technologies Institute. The location of Advanced Natural Language Processing(11711) is Pittsburgh, Pennsylvania. The room of Advanced Natural Language Processing(11711) is TEP 1403. The course happens in TR, from 12:30PM to 01:

# Batch Generation

In [ ]:
# # each question takes around 8-10s
# # 100 question takes about 15-20 minutes
question_path = "../data/test/TotalQ.txt" #"/content/drive/MyDrive/CMU/Junior/11711/chuangjl/data/test/TotalQ.txt" #
answer_list = []
embed_name = EMBEDDING_MODEL_NAME.split(sep = "/")[-1]
model_name = READER_MODEL_NAME.split(sep = "/")[-1]
print(embed_name)
print(model_name)
answer_path = f"../exp_results/{embed_name}_{model_name}.txt"
print(answer_path)
# -1 means all questions
# set the number of questions you want to set
number_of_question = -1
with open(question_path, "r", encoding="utf-8") as f:
    question_count = 0
    for line in tqdm(f):
        if question_count == number_of_question:
            break
        question = line
        final_prompt, answer = rag(question = question,
                                   llm = reader_pipeline,
                                   database = database,
                                   rerank_model = rerank_model,
                                   print_output = False)
        print(question)
        answer = answer.replace("\n", "")
        answer = answer.strip()
        print(answer)
        answer_list.append(answer)
        question_count += 1

with open(answer_path, "w+", encoding="utf-8") as f:
    for answer in answer_list:
        f.write(answer + "\n")

# Closed-book Experiment

In [ ]:
# question_path = "../data/test/TotalQ.txt" #"/content/drive/MyDrive/CMU/Junior/11711/chuangjl/data/test/TotalQ.txt" #
# answer_list = []
# embed_name = EMBEDDING_MODEL_NAME.split(sep = "/")[-1]
# model_name = READER_MODEL_NAME.split(sep = "/")[-1]
# print(embed_name)
# print(model_name)
# answer_path = f"../exp_results/closed_book_{model_name}.txt"
# print(answer_path)
# # -1 means all questions
# # set the number of questions you want to set
# number_of_question = -1
# with open(question_path, "r", encoding="utf-8") as f:
#     question_count = 0
#     for line in tqdm(f):
#         if question_count == number_of_question:
#             break
#         question = line
#         answer = reader_pipeline(question, pad_token_id=reader_pipeline.tokenizer.eos_token_id)[0]["generated_text"]
#         print(question)
#         answer = answer.replace("\n", "")
#         answer = answer.strip()
#         print(answer)
#         answer_list.append(answer)
#         question_count += 1

# with open(answer_path, "w+", encoding="utf-8") as f:
#     for answer in answer_list:
#         f.write(answer + "\n")

# Evaluation (Precision, Recall, F1)

In [ ]:
def Metric(output, ref):
  ref = ref.lower().split()
  macGen = output.lower().split()
  setRef = set(ref)
  setGen = set(macGen)
  precision = 0
  setRefNew = set([])
  setGenNew = set([])
  for ele in setRef:
    x = ele.replace(",", "")
    x = x.replace(".", "")
    x = x.replace("?", "")
    x = x.replace("!", "")
    x = x.replace("\"", "")
    x = x.replace("'", "")
    x = x.replace(" ", "")
    x = x.replace("(", "")
    x = x.replace(")", "")
    setRefNew.add(x.strip())

  for ele in setGen:
    x = ele.replace(",", "")
    x = x.replace(".", "")
    x = x.replace("?", "")
    x = x.replace("!", "")
    x = x.replace("\"", "")
    x = x.replace("'", "")
    x = x.replace(" ", "")
    x = x.replace("(", "")
    x = x.replace(")", "")
    setGenNew.add(x.strip())

  setGen = setGenNew
  setRef = setRefNew
  # print(setGen, setRef)

  if len(setGen) != 0:
    precision = len(setRef.intersection(setGen)) / len(setGen)
  recall = 0
  if len(setRef) != 0:
    recall = len(setRef.intersection(setGen)) / len(setRef)
  f1 = 0
  if precision + recall != 0:
    f1 = 2 * (precision * recall) / (precision + recall)
  return recall, precision, f1

gen_ans_path = answer_path
ref_ans_path = "../data/test/TotalA.txt"

with open(gen_ans_path, 'r', encoding='utf-8') as gen, open(ref_ans_path, 'r', encoding='utf-8') as ref:
  i = 0
  rsSum = 0
  psSum = 0
  f1Sum = 0
  for line1, line2 in zip(gen, ref):
    r, p, f = Metric(line1, line2)
    rsSum += r
    psSum += p
    f1Sum += f
    i += 1
  print("Precision=", psSum/(i))
  print("Recall=", rsSum/(i))
  print("F1=", f1Sum/(i))

RecallVal= 0.18464209401709406
Precision= 0.008956096172204216
F1= 0.016258215307389395
